In [ ]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

def evaluate_model_with_per_example_progress(dataset, model, tokenizer, context_length=128, print_every=100):
    model.eval()

    total_loss = 0.0
    total_predictions = 0
    correct_predictions = 0

    # Tokenize and process dataset
    def tokenize_function(example):
        return tokenizer(example['text'], truncation=True, padding='max_length', max_length=context_length)

    tokenized_dataset = dataset.map(tokenize_function, batched=False)

    # Convert the dataset to PyTorch tensors
    tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    # Track total number of examples
    total_examples = len(tokenized_dataset)

    with torch.no_grad():
        for i, example in enumerate(tokenized_dataset):
            input_ids = example['input_ids'].unsqueeze(0)  # Add batch dimension
            attention_mask = example['attention_mask'].unsqueeze(0)

            # Move tensors to the same device as the model
            input_ids = input_ids.to(model.device)
            attention_mask = attention_mask.to(model.device)

            # Get model output
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = outputs.logits

            # Shift input_ids and labels by one to predict next token
            shift_logits = predictions[..., :-1, :].contiguous()
            shift_labels = input_ids[..., 1:].contiguous()

            # Compute the loss
            loss_fct = torch.nn.CrossEntropyLoss(reduction='sum')
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            total_loss += loss.item()
            total_predictions += shift_labels.numel()

            # Calculate accuracy: compare the predicted token with the actual next token
            predicted_tokens = torch.argmax(shift_logits, dim=-1)
            correct_predictions += (predicted_tokens == shift_labels).sum().item()

            # Print progress every N examples
            if (i + 1) % print_every == 0:
                current_perplexity = torch.exp(torch.tensor(total_loss / total_predictions)).item()
                current_accuracy = correct_predictions / total_predictions
                print(f"Processed {i + 1}/{total_examples} examples. Current Perplexity: {current_perplexity:.2f}, Accuracy: {current_accuracy * 100:.2f}%")

    # Final metrics calculation
    perplexity = torch.exp(torch.tensor(total_loss / total_predictions)).item()
    accuracy = correct_predictions / total_predictions

    return accuracy, perplexity

# Load the model and tokenizer
model_name = "microsoft/MiniLM-L12-H384-uncased"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the PTB dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split='test')

# Evaluate the model
context_length = 32  # Set the desired fixed context length
accuracy, perplexity = evaluate_model_with_per_example_progress(dataset, model, tokenizer, context_length, print_every=100)
print(f"MiniLM Perplexity on PTB dataset with context length {context_length}: {perplexity:.2f}")
print(f"MiniLM Accuracy on PTB dataset with context length {context_length}: {accuracy * 100:.2f}%")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Processed 100/4358 examples. Current Perplexity: 41358.08, Accuracy: 0.03%
Processed 200/4358 examples. Current Perplexity: 40871.80, Accuracy: 0.02%
Processed 300/4358 examples. Current Perplexity: 40906.27, Accuracy: 0.01%
Processed 400/4358 examples. Current Perplexity: 41447.79, Accuracy: 0.01%
Processed 500/4358 examples. Current Perplexity: 41369.64, Accuracy: 0.01%
Processed 600/4358 examples. Current Perplexity: 41215.15, Accuracy: 0.01%
Processed 700/4358 examples. Current Perplexity: 41491.37, Accuracy: 0.00%
Processed 800/4358 examples. Current Perplexity: 41539.16, Accuracy: 0.00%
Processed 900/4358 examples. Current Perplexity: 41165.61, Accuracy: 0.00%
Processed 1000/4358 examples. Current Perplexity: 41250.86, Accuracy: 0.00%
Processed 1100/4358 examples. Current Perplexity: 41112.80, Accuracy: 0.00%
Processed 1200/4358 examples. Current Perplexity: 41022.26, Accuracy: 0.00%
Processed 1300/4358 examples. Current Perplexity: 41051.53, Accuracy: 0.00%
Processed 1400/4358 e

In [ ]:
#wiki-text 103 raw test set
#2-> 58385.12
#4-> 43948.05
#8-> 57331.96
#16->73945.55
#32->33567.81
#64->68098.13